문자를 하나하나떼기때문에 중복제외하고 문자 집단을 만드록
케릭터와 인덱스값으로 딕셔너리 만드록

히든 사이즈를 25로하는데 중복문자가 25라서

한번에 다 읽는게 아니라 시퀀스랭스를 10
히든은 25


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

더 긴 문자 데이터로 학습

In [28]:
sentence= ("""if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work,
but rather teach them to long for the endless immensity of the sea.""")
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work,\nbut rather teach them to long for the endless immensity of the sea."

In [29]:
char_set= list(set(sentence)) # 중복을제거한문자집합생성
char_dic= {c: i for i, c in enumerate(char_set)} # 각문자에정수인코딩
print(char_dic)
print(char_set)

{'i': 0, 'p': 1, '.': 2, 't': 3, 'w': 4, 'g': 5, 'd': 6, 'c': 7, 'n': 8, 'y': 9, 'f': 10, 'u': 11, 's': 12, 'a': 13, ' ': 14, 'r': 15, 'm': 16, 'b': 17, 'h': 18, 'o': 19, '\n': 20, 'k': 21, 'l': 22, 'e': 23, ',': 24, "'": 25}
['i', 'p', '.', 't', 'w', 'g', 'd', 'c', 'n', 'y', 'f', 'u', 's', 'a', ' ', 'r', 'm', 'b', 'h', 'o', '\n', 'k', 'l', 'e', ',', "'"]


In [30]:
dic_size= len(char_dic)
print(' 문자집합의크기: {}'.format(dic_size))

 문자집합의크기: 26


In [31]:
# 하이퍼파라미터정
hidden_size= dic_size#26
sequence_length= 10# 임의의숫자지정
learning_rate= 0.1
print(hidden_size)
print(sequence_length)
print(learning_rate)

26
10
0.1


abcdefg

x 값
abc
y 값
bcd

한칸 물려

한칸씩 넘어가니까 sequence_length를 빼줘야지 삐꾸가 안나?
슬라이딩윈도우

In [32]:
# 데이터구성 - 너무 긴 문장을 끊어서 너무 긴 문장 -sequence_length 한번더 읽을걸 남겨줘야하니 빼
x_data= []
y_data= []
for i in range(0, len(sentence) -sequence_length):
    x_str= sentence[i:i+ sequence_length]
    y_str= sentence[i+1: i+sequence_length+1]
    # print(i, x_str, '->', y_str)
    x_data.append([char_dic[c] for c in x_str]) # x strto index
    y_data.append([char_dic[c] for c in y_str]) # y strto index
    print(f'{x_str}:{y_str}')

if you wan:f you want
f you want: you want 
 you want :you want t
you want t:ou want to
ou want to:u want to 
u want to : want to b
 want to b:want to bu
want to bu:ant to bui
ant to bui:nt to buil
nt to buil:t to build
t to build: to build 
 to build :to build a
to build a:o build a 
o build a : build a s
 build a s:build a sh
build a sh:uild a shi
uild a shi:ild a ship
ild a ship:ld a ship,
ld a ship,:d a ship, 
d a ship, : a ship, d
 a ship, d:a ship, do
a ship, do: ship, don
 ship, don:ship, don'
ship, don':hip, don't
hip, don't:ip, don't 
ip, don't :p, don't d
p, don't d:, don't dr
, don't dr: don't dru
 don't dru:don't drum
don't drum:on't drum 
on't drum :n't drum u
n't drum u:'t drum up
't drum up:t drum up 
t drum up : drum up p
 drum up p:drum up pe
drum up pe:rum up peo
rum up peo:um up peop
um up peop:m up peopl
m up peopl: up people
 up people:up people 
up people :p people t
p people t: people to
 people to:people tog
people tog:eople toge
eople toge:ople toget
ople toget

char_dic[c] 했으니 수치로 붙는다.

In [33]:
print(x_data)
print(y_data)

[[0, 10, 14, 9, 19, 11, 14, 4, 13, 8], [10, 14, 9, 19, 11, 14, 4, 13, 8, 3], [14, 9, 19, 11, 14, 4, 13, 8, 3, 14], [9, 19, 11, 14, 4, 13, 8, 3, 14, 3], [19, 11, 14, 4, 13, 8, 3, 14, 3, 19], [11, 14, 4, 13, 8, 3, 14, 3, 19, 14], [14, 4, 13, 8, 3, 14, 3, 19, 14, 17], [4, 13, 8, 3, 14, 3, 19, 14, 17, 11], [13, 8, 3, 14, 3, 19, 14, 17, 11, 0], [8, 3, 14, 3, 19, 14, 17, 11, 0, 22], [3, 14, 3, 19, 14, 17, 11, 0, 22, 6], [14, 3, 19, 14, 17, 11, 0, 22, 6, 14], [3, 19, 14, 17, 11, 0, 22, 6, 14, 13], [19, 14, 17, 11, 0, 22, 6, 14, 13, 14], [14, 17, 11, 0, 22, 6, 14, 13, 14, 12], [17, 11, 0, 22, 6, 14, 13, 14, 12, 18], [11, 0, 22, 6, 14, 13, 14, 12, 18, 0], [0, 22, 6, 14, 13, 14, 12, 18, 0, 1], [22, 6, 14, 13, 14, 12, 18, 0, 1, 24], [6, 14, 13, 14, 12, 18, 0, 1, 24, 14], [14, 13, 14, 12, 18, 0, 1, 24, 14, 6], [13, 14, 12, 18, 0, 1, 24, 14, 6, 19], [14, 12, 18, 0, 1, 24, 14, 6, 19, 8], [12, 18, 0, 1, 24, 14, 6, 19, 8, 25], [18, 0, 1, 24, 14, 6, 19, 8, 25, 3], [0, 1, 24, 14, 6, 19, 8, 25, 3, 14], [

In [34]:
x_one_hot=[np.eye(dic_size)[x] for x in x_data] # x 데이터는원-핫인코딩
X= torch.FloatTensor(x_one_hot)
Y= torch.LongTensor(y_data)
print(' 훈련데이터의크기: {}'.format(X.shape))   # torch.Size([170, 10, 25])
print(' 레이블의크기: {}'.format(Y.shape))       # torch.Size([170, 10])

print(X.size(), Y.size())


 훈련데이터의크기: torch.Size([170, 10, 26])
 레이블의크기: torch.Size([170, 10])
torch.Size([170, 10, 26]) torch.Size([170, 10])


In [35]:
# print(x_one_hot)

print(X) #3차원
print(Y) #2차원 롱텐서

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0., 

In [36]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers): # 현재hidden_size는dic_size와같음
    super(Net, self).__init__()
    self.rnn= torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc= torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status= self.rnn(x)
    x= self.fc(x)
    return x

In [37]:
net= Net(dic_size, hidden_size, 2) # 이번에는층을두개쌓습니다.
criterion= torch.nn.CrossEntropyLoss()
optimizer= optim.Adam(net.parameters(), learning_rate)

In [38]:
output= net(X) # 초기예측값
aa=output.argmax(dim=2)
print(aa)
print(output.shape) # 3차원텐서
print(output.view(-1, dic_size).shape) # 2차원텐서로변환.

tensor([[22, 18, 18,  ..., 18, 18,  0],
        [18, 18, 18,  ..., 18,  0, 18],
        [13, 18,  3,  ...,  0, 18, 18],
        ...,
        [ 3,  3, 18,  ...,  0,  0, 18],
        [13, 18, 18,  ...,  0, 18,  0],
        [13, 18, 18,  ..., 18,  0, 18]])
torch.Size([170, 10, 26])
torch.Size([1700, 26])


In [39]:
# 훈련 및 예측
for i in range(1000):
  optimizer.zero_grad()
  outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
  loss=criterion(outputs.view(-1, dic_size), Y.view(-1)) 
  #한번에 10개씩 읽은거 데이터 갯수와 시퀀스 갯수 종류 곱하고 1차원 y 값과 손실함수 구해.?
  loss.backward()
  optimizer.step()
  # results의 텐서 크기는 (170, 10)
  results = outputs.argmax(dim=2)
  pred_str = ""
  for j, result in enumerate(results):
    if j == 0: # 처음에는 예측결과를 전부 가져오지만
        pred_str += ''.join([char_set[t] for t in result])
    else: # 그 다음에는 마지막 글자만 반복 추가
        pred_str += char_set[result[-1]]
  print(pred_str)

lhhhtiihhihhhhihiiihihhhtihdihdiiihhiiiiihihtdhiiihhtidhhiihhihdiiiidhhhahihihihdiiihhtiihihhhiiihdhihihihihhiiiitidahhdhiiihdhiiihhiiihhiitihihhiihhiiiihiiiiiidaiihihdihhhhhiihih
labgbkbkagabbrkakbkrbblarkalbrbklbbrkblbkrbabrkbabbrlbbbarlablallbarbbkllbbbllbbllbbbalgbbgbbbarbaaglklaglbklkbklababklaablbaallbbaaarkabkbllgbbbkbaaaaaglbkglbklaaglbbrkbkbabaakaa
t tdt tdt  ddd ddd dddtd dm tddd dtdd dd  tdmd dd dddddd dddd  ddddd ddd  dh  ddd tdd   ddt ddd ttd dddtd hdd  dd md dd dd  ddd ddd  ddd dddd dd  ddd dd  tdd tdd ddd dd tdd dt dd 
    o    o     o     oo      o     o     o        o                   o     o        o      o     o     o         o                       o                              o         
aha
d  h,nla,a,.n,aaa,,.a,,,,,,.aha,,s,a,,,,a,a,,,,,,a,a,,,,a,ah,a,a,,,,.aa,,,a,,nd a,.oaha,,,,,aa,,dha,,o,,,nd a,.naha,,h,,,,hah,,,,,a,a,.,,aaa,,,hah,h,,,a,,,,,waa,,nawa,,hah,h,,
anaodoaothtaododeanaanaoaodaaraotatoaadaotar orasareotoaotandodhsraraanretandaaodoahaodaandadedraoto

In [ ]:
for step in range(500):
  optimizer.zero_grad()
  output=model(X)
  loss=loss_func(output, Y.view(-1)) #y는 항상 1차원으로 만들어
  loss.backward()
  optimizer.step()

  pred=output.softmax(-1).argmax(-1).tolist() #아웃풋을 소프트맥스, 아그맥스하고 투 리스트한 것을 pred
  if step%10==0:
    # print(pred)
    print(f'{step}, {pred},\n {' '.join(['IF']+decode(pred))}') #단어니까 ' ' 띄워, IF는 정답 포함안되있어서.

